In [ ]:
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit-machine-learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
  Attempting uninstall: qiskit
    Found existing installation: qiskit 2.2.3
    Uninstalling qiskit-2.2.3:
      Successfully uninstalled qis

In [ ]:
# Cell 1: Imports and tunable parameters
import os, time
import numpy as np
import pandas as pd
from collections import Counter

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Repro
SEED = 42
np.random.seed(SEED)

# Parameters (set these to match your quantum notebook)
PCA_COMPONENTS = 12        # same as quantum PCA_COMPONENTS
ANOMALY_SUBSET_SIZE = 2000 # number of normal samples for one-class (same as quantum experiment)
NUM_CLASSES = 4            # total classes including 'normal' (same as quantum)
PER_CLASS = 400            # training samples per class for multiclass (same as quantum)
TEST_SIZE = 1000            # number of test samples to evaluate
SAVE_XTRAIN = "X_train_pca.npy"
SAVE_XTEST  = "X_test_pca.npy"
SAVE_DFTRAIN = "df_train.pkl"
SAVE_DFTEST  = "df_test.pkl"

print("Parameters:", "PCA=", PCA_COMPONENTS, "ANOMALY_SUBSET=", ANOMALY_SUBSET_SIZE,
      "NUM_CLASSES=", NUM_CLASSES, "PER_CLASS=", PER_CLASS, "TEST_SIZE=", TEST_SIZE)


Parameters: PCA= 12 ANOMALY_SUBSET= 2000 NUM_CLASSES= 4 PER_CLASS= 400 TEST_SIZE= 1000


In [ ]:
# Cell 2: Load raw NSL-KDD train/test files
TRAIN_RAW = "KDDTrain+.txt"
TEST_RAW  = "KDDTest+.txt"

columns = [
    'duration','protocol_type','service','flag','src_bytes','dst_bytes',
    'land','wrong_fragment','urgent','hot','num_failed_logins','logged_in',
    'num_compromised','root_shell','su_attempted','num_root','num_file_creations',
    'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login',
    'count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate',
    'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count',
    'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate',
    'dst_host_rerror_rate','dst_host_srv_rerror_rate','label','difficulty'
]

print("Loading raw files...")
df_train = pd.read_csv(TRAIN_RAW, names=columns)
df_test  = pd.read_csv(TEST_RAW,  names=columns)
print("Loaded. Train shape:", df_train.shape, "Test shape:", df_test.shape)


Loading raw files...
Loaded. Train shape: (125973, 43) Test shape: (22544, 43)


In [ ]:
# Cell 3: Clean label column, inspect class counts
# Make sure labels are strings and normalized
df_train['label'] = df_train['label'].astype(str).str.strip().str.lower().str.rstrip('.')
df_test['label']  = df_test['label'].astype(str).str.strip().str.lower().str.rstrip('.')

print("Unique labels (train):", df_train['label'].unique()[:30])
print("Top label counts (train):")
print(df_train['label'].value_counts().head(12))

n_normal_train = (df_train['label'] == 'normal').sum()
print("Number of normal samples in train:", n_normal_train)

Unique labels (train): ['normal' 'neptune' 'warezclient' 'ipsweep' 'portsweep' 'teardrop' 'nmap'
 'satan' 'smurf' 'pod' 'back' 'guess_passwd' 'ftp_write' 'multihop'
 'rootkit' 'buffer_overflow' 'imap' 'warezmaster' 'phf' 'land'
 'loadmodule' 'spy' 'perl']
Top label counts (train):
label
normal          67343
neptune         41214
satan            3633
ipsweep          3599
portsweep        2931
smurf            2646
nmap             1493
back              956
teardrop          892
warezclient       890
pod               201
guess_passwd       53
Name: count, dtype: int64
Number of normal samples in train: 67343


In [ ]:
# Cell 4: One-hot encode categorical columns, align train/test columns, scale numeric data
# We'll save processed DataFrames and scaled arrays for reuse by Quantum notebook
print("Preprocessing: one-hot encoding categorical columns and scaling.")

# Drop 'difficulty' column for features
df_train_proc = df_train.drop(columns=['difficulty']).copy()
df_test_proc  = df_test.drop(columns=['difficulty']).copy()

# Detect categorical columns (object dtype) except 'label'
cat_cols = df_train_proc.select_dtypes(include=['object']).columns.tolist()
cat_cols = [c for c in cat_cols if c != 'label']
print("Categorical columns to encode:", cat_cols)

# One-hot encode categorical columns
df_train_proc = pd.get_dummies(df_train_proc, columns=cat_cols)
df_test_proc  = pd.get_dummies(df_test_proc,  columns=cat_cols)

# Align test columns to train columns
df_test_proc = df_test_proc.reindex(columns=df_train_proc.columns, fill_value=0)

# Extract labels and features
y_train_full = df_train_proc['label'].astype(str).values
y_test_full  = df_test_proc['label'].astype(str).values
X_train_df = df_train_proc.drop(columns=['label']).copy()
X_test_df  = df_test_proc.drop(columns=['label']).copy()

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_df)
X_test_scaled  = scaler.transform(X_test_df)

print("Preprocessing complete. Shapes:", X_train_scaled.shape, X_test_scaled.shape)

# Save processed DF's for quantum notebook to load easily
df_train.to_pickle(SAVE_DFTRAIN)
df_test.to_pickle(SAVE_DFTEST)
print("Saved df_train/df_test pkl for reuse by Quantum notebook.")

Preprocessing: one-hot encoding categorical columns and scaling.
Categorical columns to encode: ['protocol_type', 'service', 'flag']
Preprocessing complete. Shapes: (125973, 122) (22544, 122)
Saved df_train/df_test pkl for reuse by Quantum notebook.


In [ ]:
# Cell 5: PCA reduction to PCA_COMPONENTS and save arrays for Notebook B
from sklearn.decomposition import PCA

pca = PCA(n_components=PCA_COMPONENTS, random_state=SEED)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca  = pca.transform(X_test_scaled)

# Save arrays for Quantum notebook to load
np.save(SAVE_XTRAIN, X_train_pca)
np.save(SAVE_XTEST, X_test_pca)
print("PCA done. Shapes:", X_train_pca.shape, X_test_pca.shape)
print("Saved X_train_pca.npy and X_test_pca.npy for reuse.")
print("Explained variance ratio (sum):", pca.explained_variance_ratio_.sum())

PCA done. Shapes: (125973, 12) (22544, 12)
Saved X_train_pca.npy and X_test_pca.npy for reuse.
Explained variance ratio (sum): 0.3136342833926865


In [ ]:
# Cell 6: Classical One-Class SVM baseline — train on the SAME normal subset you'll use for quantum
print("Building One-Class SVM baseline using the same normal subset size:", ANOMALY_SUBSET_SIZE)

# Collect all normal samples from PCA data
normal_mask = (y_train_full == 'normal')
X_train_normal_all = X_train_pca[normal_mask]
n_normals = X_train_normal_all.shape[0]
print("Available normal samples:", n_normals)

# Choose subset for training (reproducible)
subset_size = min(ANOMALY_SUBSET_SIZE, n_normals)
np.random.seed(SEED)
selected_idx_normals = np.random.choice(n_normals, subset_size, replace=False)
X_train_subset_normal = X_train_normal_all[selected_idx_normals]

# Prepare test set (same test selection strategy as quantum notebook — first TEST_SIZE)
test_size = min(TEST_SIZE, X_test_pca.shape[0])
X_test_eval = X_test_pca[:test_size]
# make binary labels: 0=normal, 1=attack
y_test_eval = (y_test_full[:test_size] != 'normal').astype(int)

print("Train subset shape (normal):", X_train_subset_normal.shape)
print("Test eval shape:", X_test_eval.shape, "with", np.sum(y_test_eval==1), "attacks in test subset.")

# Train classical One-Class SVM (RBF)
t0 = time.time()
oc_classical = OneClassSVM(kernel='rbf', gamma='scale', nu=0.1)
oc_classical.fit(X_train_subset_normal)
t1 = time.time()

# Predict and evaluate
t2 = time.time()
y_pred_c = oc_classical.predict(X_test_eval)   # +1 inlier, -1 outlier
y_pred_c = np.where(y_pred_c == 1, 0, 1)
t3 = time.time()

print(f"Classical One-Class SVM: train_time={t1-t0:.3f}s predict_time={t3-t2:.3f}s")
print("Accuracy (classical one-class):", accuracy_score(y_test_eval, y_pred_c))
print("Confusion matrix:\n", confusion_matrix(y_test_eval, y_pred_c))
print("Classification report:\n", classification_report(y_test_eval, y_pred_c, zero_division=0))

Building One-Class SVM baseline using the same normal subset size: 2000
Available normal samples: 67343
Train subset shape (normal): (2000, 12)
Test eval shape: (1000, 12) with 548 attacks in test subset.
Classical One-Class SVM: train_time=0.032s predict_time=0.013s
Accuracy (classical one-class): 0.84
Confusion matrix:
 [[404  48]
 [112 436]]
Classification report:
               precision    recall  f1-score   support

           0       0.78      0.89      0.83       452
           1       0.90      0.80      0.84       548

    accuracy                           0.84      1000
   macro avg       0.84      0.84      0.84      1000
weighted avg       0.85      0.84      0.84      1000



In [ ]:
# Cell 7: Classical multiclass baseline – build balanced subset identical to quantum multiclass setup
print("Preparing balanced multiclass training subset: NUM_CLASSES =", NUM_CLASSES, "PER_CLASS =", PER_CLASS)

labels_all = df_train['label'].astype(str).str.strip().str.lower().to_numpy()
unique, counts = np.unique(labels_all, return_counts=True)
label_counts = sorted(zip(unique, counts), key=lambda x: x[1], reverse=True)
print("Top labels (train):", label_counts[:10])

# Select classes: always include 'normal' then top attacks (same logic as Quantum)
classes = ['normal'] if 'normal' in unique else []
for lbl, _ in label_counts:
    if lbl == 'normal': continue
    if len(classes) >= NUM_CLASSES:
        break
    classes.append(lbl)
classes = classes[:NUM_CLASSES]
print("Selected classes for multiclass baseline:", classes)

# Build indices per class
np.random.seed(SEED)
indices_per_class = []
for cls in classes:
    cls_idxs = np.where(labels_all == cls)[0]
    if len(cls_idxs) == 0:
        raise RuntimeError(f"No samples found for class '{cls}'")
    take = min(PER_CLASS, len(cls_idxs))
    chosen = np.random.choice(cls_idxs, take, replace=False)
    indices_per_class.append(chosen)

train_idx_multi = np.concatenate(indices_per_class)
X_train_multi = X_train_pca[train_idx_multi]
y_train_multi = labels_all[train_idx_multi]

print("Multiclass train shape:", X_train_multi.shape)
print("Per-class counts:", Counter(y_train_multi))

Preparing balanced multiclass training subset: NUM_CLASSES = 4 PER_CLASS = 400
Top labels (train): [('normal', np.int64(67343)), ('neptune', np.int64(41214)), ('satan', np.int64(3633)), ('ipsweep', np.int64(3599)), ('portsweep', np.int64(2931)), ('smurf', np.int64(2646)), ('nmap', np.int64(1493)), ('back', np.int64(956)), ('teardrop', np.int64(892)), ('warezclient', np.int64(890))]
Selected classes for multiclass baseline: ['normal', 'neptune', 'satan', 'ipsweep']
Multiclass train shape: (1600, 12)
Per-class counts: Counter({'normal': 400, 'neptune': 400, 'satan': 400, 'ipsweep': 400})


In [ ]:
# Cell 8: Train classical multiclass classifiers on the SAME training subset and evaluate on same test
# Prepare test set (we choose first TEST_SIZE test samples; ensure those include many classes)
test_n = min(TEST_SIZE, X_test_pca.shape[0])
X_test_multi_eval = X_test_pca[:test_n]
y_test_multi_eval = df_test['label'].astype(str).str.strip().str.lower().to_numpy()[:test_n]

print("Test multi eval shape:", X_test_multi_eval.shape, "unique labels in test sample:", np.unique(y_test_multi_eval)[:10])

# 1) SVC (RBF)
t0 = time.time()
svc = SVC(kernel='rbf', gamma='scale', class_weight='balanced')
svc.fit(X_train_multi, y_train_multi)
t1 = time.time()
y_pred_svc = svc.predict(X_test_multi_eval)
t2 = time.time()
print(f"SVC train_time={t1-t0:.2f}s predict_time={t2-t1:.2f}s")
print("SVC Accuracy:", accuracy_score(y_test_multi_eval, y_pred_svc))
print("SVC Confusion Matrix (selected classes):\n", confusion_matrix(y_test_multi_eval, y_pred_svc, labels=classes))
print("SVC Classification Report:\n", classification_report(y_test_multi_eval, y_pred_svc, labels=classes, zero_division=0))

# 2) Random Forest
t0 = time.time()
rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=SEED)
rf.fit(X_train_multi, y_train_multi)
t1 = time.time()
y_pred_rf = rf.predict(X_test_multi_eval)
t2 = time.time()
print(f"RF train_time={t1-t0:.2f}s predict_time={t2-t1:.2f}s")
print("RF Accuracy:", accuracy_score(y_test_multi_eval, y_pred_rf))
print("RF Confusion Matrix (selected classes):\n", confusion_matrix(y_test_multi_eval, y_pred_rf, labels=classes))
print("RF Classification Report:\n", classification_report(y_test_multi_eval, y_pred_rf, labels=classes, zero_division=0))

Test multi eval shape: (1000, 12) unique labels in test sample: ['apache2' 'back' 'buffer_overflow' 'guess_passwd' 'httptunnel' 'ipsweep'
 'mailbomb' 'mscan' 'multihop' 'named']
SVC train_time=0.02s predict_time=0.01s
SVC Accuracy: 0.659
SVC Confusion Matrix (selected classes):
 [[406   1  41   4]
 [  0 215   0   0]
 [  1   0  32   0]
 [  0   0   0   6]]
SVC Classification Report:
               precision    recall  f1-score   support

      normal       0.69      0.90      0.78       452
     neptune       0.82      1.00      0.90       215
       satan       0.35      0.97      0.51        33
     ipsweep       0.11      1.00      0.20         6

   micro avg       0.66      0.93      0.77       706
   macro avg       0.49      0.97      0.60       706
weighted avg       0.71      0.93      0.80       706

RF train_time=1.49s predict_time=0.04s
RF Accuracy: 0.684
RF Confusion Matrix (selected classes):
 [[434   2  13   3]
 [  0 214   1   0]
 [  3   0  30   0]
 [  0   0   0   6]]
RF C

In [ ]:
# Cell 9: Save selected indices and PCA arrays and a small summary CSV for reproducibility
os.makedirs("classical_results", exist_ok=True)

# Save PCA arrays (already saved earlier) and training indices for reproducibility
np.save(SAVE_XTRAIN, X_train_pca)
np.save(SAVE_XTEST,  X_test_pca)
np.save("classical_results/train_idx_multi.npy", train_idx_multi)
np.save("classical_results/selected_idx_normals.npy", selected_idx_normals)

# Save summary metrics to CSV
summary = {
    "quantum_PCA_components": PCA_COMPONENTS,
    "anomaly_subset_size": subset_size,
    "multiclass_num_classes": len(classes),
    "multiclass_per_class": PER_CLASS,
    "test_size": test_n
}
pd.Series(summary).to_csv("classical_results/summary_params.csv")

print("Saved artifacts to classical_results/. You can now run Notebook B (quantum) and it will load the same PCA arrays & DF pickles for 1:1 comparison.")

Saved artifacts to classical_results/. You can now run Notebook B (quantum) and it will load the same PCA arrays & DF pickles for 1:1 comparison.
